In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure the plotting routines
import pandas as pd

# Import your CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter()

# Read data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

# Check if '_id' column exists before dropping
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('AnimalShelterDashboard')

# Load and encode your image
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div([
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                    style={'height': '80px', 'margin-right': '20px'}),
            href='https://www.snhu.edu',
            target='_blank'
        ),
        html.H1('Steven Armenta Animal Shelter Dashboard', 
                style={'display': 'inline-block', 'verticalAlign': 'middle'})
    ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center'}),
    html.Hr(),
    
    # Filter widgets section
    html.Div([
        html.H3("Filter by Rescue Type:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster/Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset (Show All)', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'block', 'margin': '5px'}
        )
    ], style={'margin': '20px'}),
    
    html.Hr(),
    
    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'minWidth': '100px',
            'width': '150px',
            'maxWidth': '180px',
            'whiteSpace': 'normal'
        }
    ),
    
    html.Br(),
    html.Hr(),
    
    html.Div([
        html.Div([
            dcc.Graph(id='pie-chart')
        ], style={'width': '50%', 'display': 'inline-block', 'vertical-align': 'top'}),

        html.Div([
            html.H3("Shelter Location Map"),
            html.Div(id='map-id', style={'width': '100%', 'height': '500px'})
        ], style={'width': '50%', 'display': 'inline-block', 'vertical-align': 'top'})
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    [Output('datatable-id', 'data'),
     Output('pie-chart', 'figure')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Filter data based on selection
    if filter_type == 'water':
        filtered_df = df[
            (df['breed'].str.contains('Labrador|Newfoundland|Chesapeake', case=False, na=False)) &
            (df['sex_upon_outcome'] == 'Intact Female') &
            (df['age_upon_outcome_in_weeks'].between(26, 156))
        ]
    elif filter_type == 'mountain':
        filtered_df = df[
            (df['breed'].str.contains('German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Husky|Rottweiler', case=False, na=False)) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'].between(26, 156))
        ]
    elif filter_type == 'disaster':
        filtered_df = df[
            (df['breed'].str.contains('Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler', case=False, na=False)) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'].between(20, 300))
        ]
    else:
        filtered_df = df  # Reset shows all data
    
    # Create pie chart of breed distribution
    breed_counts = filtered_df['breed'].value_counts().nlargest(5)
    pie_data = pd.DataFrame({
        'breed': breed_counts.index,
        'count': breed_counts.values
    })
    
    pie_fig = px.pie(pie_data, names='breed', values='count', title='Top 5 Breed Distribution')
    pie_fig.update_traces(textinfo='percent+label')
    
    return filtered_df.to_dict('records'), pie_fig

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [
        {
            'if': {'column_id': i},
            'background_color': '#D2F3FF'
        } for i in selected_columns
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if viewData is None or selected_rows is None or len(selected_rows) == 0:
        # Default map when no row is selected
        return dl.Map(style={'width': '100%', 'height': '500px'},
                     center=[30.75, -97.48], zoom=10, 
                     children=[dl.TileLayer()])
    
    dff = pd.DataFrame.from_dict(viewData)
    row_idx = selected_rows[0]
    
    if dff.empty or row_idx >= len(dff):
        return dl.Map(style={'width': '100%', 'height': '500px'},
                     center=[30.75, -97.48], zoom=10, 
                     children=[dl.TileLayer()])
    
    # Find the latitude and longitude columns
    lat_col = next((col for col in dff.columns if 'lat' in col.lower()), None)
    lon_col = next((col for col in dff.columns if 'lon' in col.lower()), None)
    
    # If location columns are not found, use default map
    if lat_col is None or lon_col is None:
        return dl.Map(style={'width': '100%', 'height': '500px'},
                     center=[30.75, -97.48], zoom=10, 
                     children=[dl.TileLayer()])
    
    # Get name column - assuming 'name' is in column name
    name_col = next((col for col in dff.columns if 'name' in col.lower()), None)
    animal_name = dff.iloc[row_idx][name_col] if name_col else "Unknown"
    
    # Get animal type/breed
    breed_col = next((col for col in dff.columns if 'breed' in col.lower()), None)
    animal_breed = dff.iloc[row_idx][breed_col] if breed_col else "Unknown"
    
    return dl.Map(
        style={'width': '100%', 'height': '500px'},
        center=[dff.iloc[row_idx][lat_col], dff.iloc[row_idx][lon_col]], 
        zoom=15, 
        children=[
            dl.TileLayer(),
            dl.Marker(
                position=[dff.iloc[row_idx][lat_col], dff.iloc[row_idx][lon_col]],
                children=[
                    dl.Tooltip(animal_breed),
                    dl.Popup([
                        html.H4("Animal Details"),
                        html.P(f"Name: {animal_name}"),
                        html.P(f"Breed: {animal_breed}")
                    ])
                ]
            )
        ]
    )

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
else:
    app.run_server(debug=True)

TypeError: an integer is required (got type NoneType)